# AI for Predictive Maintenance in Manufacturing

**Copyright (c) 2026 Shrikara Kaudambady. All rights reserved.**

This notebook demonstrates how to predict the Remaining Useful Life (RUL) of a machine using deep learning. We will use the NASA Turbofan Engine Degradation dataset to train a Long Short-Term Memory (LSTM) network. The model learns to predict the RUL based on a sequence of historical sensor readings from the engine.

### 1. Setup and Library Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

### 2. Load and Prepare the Data
We will load the NASA Turbofan dataset (FD001). The dataset is already split into training and test sets. The training data runs until failure, while the test data stops at some point before failure. We first need to calculate the Remaining Useful Life (RUL) for the training data.

In [ ]:
# Define column names for the dataset
columns = ['engine_id', 'cycle', 'setting1', 'setting2', 'setting3'] + [f's{i}' for i in range(1, 22)]

# Load training data
train_df = pd.read_csv('https://raw.githubusercontent.com/dayjournal/CMAPSS/master/train_FD001.txt', sep='\\s+', header=None, names=columns)

# Calculate RUL for the training data
max_cycles = train_df.groupby('engine_id')['cycle'].max().reset_index()
max_cycles.columns = ['engine_id', 'max_cycle']
train_df = train_df.merge(max_cycles, on='engine_id', how='left')
train_df['RUL'] = train_df['max_cycle'] - train_df['cycle']
train_df = train_df.drop('max_cycle', axis=1)

print("Training data prepared with RUL.")
train_df.head()

### 3. Feature Scaling and Sequencing
We scale the sensor data to a [0, 1] range. Then, we transform the time-series data into 'windows' or sequences, which will be the input for our LSTM model. Each sequence will consist of 50 time steps of sensor data.

In [ ]:
# Select sensor columns and scale them
sensor_cols = [f's{i}' for i in range(1, 22)]
scaler = MinMaxScaler()
train_df[sensor_cols] = scaler.fit_transform(train_df[sensor_cols])

# Create sequences from the time-series data
def create_sequences(df, sequence_length=50):
    X, y = [], []
    for engine_id in df['engine_id'].unique():
        engine_df = df[df['engine_id'] == engine_id]
        data = engine_df[sensor_cols].values
        labels = engine_df['RUL'].values
        for i in range(len(data) - sequence_length):
            X.append(data[i:i + sequence_length])
            y.append(labels[i + sequence_length - 1])
    return np.array(X), np.array(y)

SEQUENCE_LENGTH = 50
X_train, y_train = create_sequences(train_df, SEQUENCE_LENGTH)

print(f"Training data reshaped into sequences:")
print(f"X_train shape: {X_train.shape}") # (samples, timesteps, features)
print(f"y_train shape: {y_train.shape}")

### 4. Build and Train the LSTM Model
We'll build a stacked LSTM model. Stacking LSTM layers can help the model learn more complex, hierarchical patterns in the time-series data.

In [ ]:
model = Sequential([
    LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(units=50, return_sequences=False),
    Dropout(0.2),
    Dense(units=1) # Output layer for RUL prediction
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

# Train the model
print("\nTraining LSTM model...")
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, verbose=1)

# Plot training history
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.show()

### 5. Evaluate the Model on Test Data
We now load the test data, process it in the same way as the training data, and evaluate our model's performance by predicting the RUL for the test engines.

In [ ]:
# Load test data and ground truth RUL values
test_df = pd.read_csv('https://raw.githubusercontent.com/dayjournal/CMAPSS/master/test_FD001.txt', sep='\\s+', header=None, names=columns)
truth_df = pd.read_csv('https://raw.githubusercontent.com/dayjournal/CMAPSS/master/RUL_FD001.txt', sep='\\s+', header=None, names=['RUL'])

# Scale the test data using the same scaler from training
test_df[sensor_cols] = scaler.transform(test_df[sensor_cols])

# For each engine in the test set, take the last 'sequence_length' cycles of data
X_test = []
for engine_id in test_df['engine_id'].unique():
    engine_df = test_df[test_df['engine_id'] == engine_id]
    data = engine_df[sensor_cols].values
    # If an engine has less than 50 cycles, pad with zeros
    if len(data) < SEQUENCE_LENGTH:
        pad = np.zeros((SEQUENCE_LENGTH - len(data), data.shape[1]))
        data = np.vstack([pad, data])
    X_test.append(data[-SEQUENCE_LENGTH:])
X_test = np.array(X_test)

y_test = truth_df['RUL'].values

# Make predictions
y_pred = model.predict(X_test).flatten()

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n--- Model Evaluation on Test Set ---")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.4f}")

# Visualize predictions vs actuals
plt.figure(figsize=(10, 10))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--', color='red', lw=2)
plt.xlabel("Actual RUL")
plt.ylabel("Predicted RUL")
plt.title(f"Predictive Maintenance: Actual vs. Predicted RUL (RMSE: {rmse:.2f})")
plt.axis('square')
plt.show()